# Creation Example
In this notebook, we demonstrate how to calculate the BNG units for an area being created

In [1]:
import jax
import jax.numpy as jnp
import pandas as pd
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.handlers import seed, trace, condition, plate
import matplotlib.pyplot as plt
import seaborn as sns # For nicer plots

# Import your JAX BNG calculation functions and constants
# Assuming your bngmetric package is installed or its path is set up
from bngmetric.core_calculations import calculate_total_bng_from_jax_arrays, calculate_batched_baseline_bng_units,calculate_baseline_bng_unit
from bngmetric.constants import (
    HABITAT_TYPE_TO_ID, CONDITION_CATEGORY_TO_ID,
    ID_TO_DISTINCTIVENESS_VALUE, CONDITION_MULTIPLIERS_MATRIX,
    
    
)

# Configure JAX for CPU for consistent behavior if not on GPU
jax.config.update('jax_platform_name', 'cpu')
jax.config.update('jax_enable_x64', True) # Enable 64-bit precision for better numerical stability

/Users/peterhurley/bngmetric/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/peterhurley/bngmetric/src/bngmetric/constants.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_enhance.rename(columns={'Start condition': 'Habitat'}, inplace=True)
/Users/peterhurley/bngmetric/src/bngmetric/constants.py:186: PerformanceWarning: indexing past lexsort depth may impact performance.
  for tc in final_enhance.loc[habitat, (start_condition,)].index:
/Users/peterhurley/bngmetric/src/bngmetric/constants.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

## Generate a toy example
Now lets create an example input from the metric, i.e. some habitats, conditions, area and strategic significance

In [2]:
# --- Example Input Data (similar to your DataFrame) ---
# Ensure these habitat names exist in your HABITAT_TYPE_TO_ID mapping
example_df = pd.DataFrame({
    'Habitat': ['Grassland - Lowland calcareous grassland', 'Grassland - Modified grassland', 'Woodland and forest - Lowland mixed deciduous woodland'],
    'Condition': ['Good', 'Moderate', 'Good'],
    'Area': [5.0, 20.0, 8.0],
    'Strategic_Significance': [1.0, 1.0, 1.15],

})


# Get reverse mapping for clear output
ID_TO_HABITAT_TYPE = {v: k for k, v in HABITAT_TYPE_TO_ID.items()}

# Convert DataFrame columns to JAX arrays using your constants mapping
classified_habitat_ids_input = jnp.array([
    HABITAT_TYPE_TO_ID[ht] for ht in example_df['Habitat'].values
])
print("Classified Habitat IDs (IDs):", classified_habitat_ids_input)
condition_ids_input = jnp.array([
    CONDITION_CATEGORY_TO_ID[c] for c in example_df['Condition'].values
])
areas_input = jnp.array(example_df['Area'].values, dtype=jnp.float64) # Use float64 for better precision
strategic_multipliers_input = jnp.array(example_df['Strategic_Significance'].values, dtype=jnp.float64)

# Print initial classification for reference
print("Initial Classified Habitats (IDs):", classified_habitat_ids_input)
print("Initial Areas:", areas_input)

Classified Habitat IDs (IDs): [13 16 89]
Initial Classified Habitats (IDs): [13 16 89]
Initial Areas: [ 5. 20.  8.]


In [3]:
from bngmetric.creation import calculate_batched_creation_bng_units,calculate_creation_bng_from_dataframe,calculate_creation_bng_unit


In [13]:
time_to_target_years=jnp.array([30,30,30],dtype=jnp.int16)
bng=calculate_batched_creation_bng_units(classified_habitat_ids_input, condition_ids_input, areas_input, strategic_multipliers_input)

In [15]:
bng.sum()

Array(101.40035987, dtype=float64)

In [12]:
calculate_creation_bng_unit(classified_habitat_ids_input[2], condition_ids_input[2], areas_input[2], strategic_multipliers_input[2])

Array(17.4873603, dtype=float64)

In [7]:
from bngmetric.core_calculations import get_distinctiveness_value, get_condition_multiplier
from bngmetric.creation import get_temporal_multiplier, get_difficulty_multiplier

distinctiveness = get_distinctiveness_value(classified_habitat_ids_input[0])
condition_m = get_condition_multiplier(classified_habitat_ids_input[0], condition_ids_input[0])
temporal_m = get_temporal_multiplier(classified_habitat_ids_input[0],condition_ids_input[0])
difficulty_m = get_difficulty_multiplier(classified_habitat_ids_input[0])

In [9]:
temporal_m,condition_m,difficulty_m,distinctiveness

(Array(0.49, dtype=float32),
 Array(3., dtype=float32),
 Array(0.33, dtype=float32),
 Array(6., dtype=float32))

In [10]:
calculate_creation_bng_from_dataframe(example_df)

101.40035986854076